# Architecture détaillée des réseaux de détection en temps réel #

Dans ce notebook, on va regarder en détail quelle est l'architecture des réseaux YOLO (v1, v2, v3), SSD, et RetinaNet afin de mieux comprendre comment fonctionne la prédiction de bounding boxes dans une image.

## YOLO v1 ##

**Architecture globale** : 24 couches de convolution suivies de 2 couches fully connected (9 couches de convolution pour Fast YOLO au lieu de 24, YOLO a une vitesse de 45 fps tandis que Fast YOLO a une vitesse de 150 fps).

**Architecture détaillée**

| Nom de la couche | Taille du noyau, du filtre ou de la couche | Nombre de filtres de convolution | Stride |
| :------------ | :------------ | :------------ | :------------ |
| Convolution Layer | 7x7 | 64 | 2 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolution Layer | 3x3 | 192 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 1x1 | 512 | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 512 | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 3x3 | 1024 | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Fully Connected Layer | 7x7x1024 | N/A | N/A |
| Fully Connected Layer | 1x4096 | N/A | N/A |
| Sortie | 7x7x30 | N/A | N/A |

Les 20 premières couches convolutives du réseau ont été entrainées sur ImageNet (BD du challenge de classification de 1000 classes). Le pré-traitement se fait avec des images de résolution 224 x 224. On ajoute ensuite 4 couches de convolution suivies de deux couches fully connected qui sont en charge de la détection d'objet. La résolution de la BD est ensuite passée à 448x448 puisque la détection d'objet a besoin d'images de meilleure résolution pour être optimales. La couche finale prédit les bounding boxes et la probabilité d'appartenance à chacune des classes. 

**Prédiction des bounding boxes**

Chaque image analysée est divisée en une grille de taille 7x7, ce qui donne 49 cellules. Pour chacune de ces cellules, 2 bounding boxes sont prédites, ce qui donne un total de 98 bounding boxes prédites par image. 
Pour chacune des cellules de la grille, le réseau ne détecte qu'un seul objet par cellule de la grille, même si deux bounding boxes sont prédites dans chacune des cellules. 

Chaque bounding boxe contient 5 éléments {(x, y, w, h) et un score de confiance de la box qui dit à quel point la box est susceptible de contenir un objet et à quel point la bounding box est précise}. Puisque chacune des cellules prédit deux bounding boxes, elle prédit 5x2 = 10 éléments. De plus, pour chacune des cellules, un score d'appartenance à chacune des 20 classes est prédit, ce qui fait que chacune des cellules de l'image renvoie 20+10 = 30 éléments. Ce qui explique que la taille de la sortie soit 7x7x30. 

La façon dont les bounding boxes sont prédites se base sur la feature map calculée par le réseau de neurones. Chacune des valeurs d'une feature map est calculée en fonction d'un receptive field. Le receptive field d'une cellule est en général plus grand que la taille de la feature map. Ce qui signifie que chacune des cellules contient des informations qui concerne les cellules avoisinantes. En considérant le contenu de la feature map et de celui des bounding boxes générées au centre de chacune des cellules, le réseau va donc pouvoir se concentrer sur chacune des bounding boxes et prédire un décalage des coordonnées du centre ainsi que la différence de hauteur et la différence de largeur. Si le décalage des coordonnées du centres prédites se situent à l'extérieur de la cellule qui a généré les bounding boxes, ce n'est pas cette cellule mais celle contenant le centre de l'objet qui sera retenue pour la prédiction finale. De cette façon il n'y a plus la contrainte d'utiliser une fenêtre glissante pour calculer des prédictions de bounding boxes qui ralenti la détection.

**Performances du réseau** : Très rapide (45 fps pour YOLO, 150 fps pour Fast YOLO mais la précision de détection n'est pas optimale). SSD est un très grand compétiteur de ce réseau pour la détection en temps réel.

Lien vers l'article : https://arxiv.org/pdf/1506.02640.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

http://christopher5106.github.io/object/detectors/2017/08/10/bounding-box-object-detectors-understanding-yolo.html

https://stats.stackexchange.com/questions/309345/how-does-the-yolo-network-create-boundaries-for-object-detection

https://stackoverflow.com/questions/50575301/yolo-object-detection-how-does-the-algorithm-predict-bounding-boxes-larger-than

https://hackernoon.com/understanding-yolo-f5a74bbc7967

https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088

https://stats.stackexchange.com/questions/295766/you-only-look-once-yolo-convolutional-neural-network

https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/

## YOLO v2 ##

**Différences avec YOLO v1** : 
- L'image est découpée en une grille de taille 13x13 au lieu de 7x7 ;
- Chacune des grilles prédit 5 bounding boxes différentes ;
- Le réseau de base est Darknet-19 au lieu de VGG-16 ;

**Architecture globale** : Au lieu d'utiliser VGG-16 pour la base du réseau, on utilise un réseau appelé Darknet-19 qui comporte 19 couches de convolution. 

**Architecture détaillée** : 

| Nom de la couche | Taille du noyau, du filtre ou de la couche | Nombre de filtres de convolution | Stride |
| :------------ | :------------ | :------------ | :------------ |
| Convolution Layer | 3x3 | 32 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolution Layer | 3x3 | 64 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Convolutional Layer | 1x1 | 64 | 1 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 1000 | 1 |
| Fully Connected Layer | 13x13x2048 | N/A | N/A |
| Fully Connected Layer | 1x4096 | N/A | N/A |
| Sortie | 13x13x45 | N/A | N/A |

Le réseau Darknet-19 est tout d'abord entraîné sur des images d'ImageNet de taille 224x224 et ensuite entraîné sur 10 épochs sur des images de taille 448x448. Cela permet au réseau de réajuster ses filtres afin qu'il puisse mieux s'adapter à des images de plus grande résolution en entrée. Cela permet d'augmenter de 4% le mAP.

Pour chacune des bounding boxes, 5 caractéristiques sont prédites (coordonnées centre, largeur, hauteur et contient ou pas un objet) ce qui fait 5x5 = 25 caractéristiques. De plus s'ajoutent à ça la probabilité d'appartenance à chacune des 20 classes ce qui donne 20+25 = 45, ce qui explique que la taille en sortie soit de 13x13x45.

**Performances du réseau** : Meilleure précision de détection que YOLO mais moins bien que SSD.

Lien vers l'article : https://pjreddie.com/media/files/papers/YOLO9000.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

https://towardsdatascience.com/training-object-detection-yolov2-from-scratch-using-cyclic-learning-rates-

http://ethereon.github.io/netscope/#/gist/d08a41711e48cf111e330827b1279c31

https://medium.com/@akarshzingade/logo-detection-using-yolov2-8cda5a68740e

## YOLO 9000 ##

**Architecture globale** : Même architecture que YOLOv2 sauf que YOLO9000 est entraîné pour détecter 9000 classes différentes.

Lien vers l'article : https://pjreddie.com/media/files/papers/YOLO9000.pdf

## YOLO v3 ##

**Différences avec YOLO v2** : 
- 53 couches de convolution au lieu de 19 ;
- Plus précis que YOLO v2 ;
- Moins rapide ;
- Détection à trois échelles différentes ;
- Meilleure détection des petits objets ;
- Un objet détecté peut être multilabel (par exemple personne et femme). 

**Architecture globale** : 53 couches de convolution et deux couches fully connected. 

**Architecture détaillée** :

| Nom de la couche | Taille du noyau, du filtre ou de la couche | Nombre de filtres de convolution | Stride |
| :------------ | :------------ | :------------ | :------------ |
| Convolution Layer | 3x3 | 32 | 1 |
| Convolution Layer | 3x3 | 64 | 2 |
| Convolution Layer | 1x1 | 32 | 1 |
| Convolution Layer | 3x3 | 64 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolution Layer | 3x3 | 128 | 2 |
| Convolutional Layer | 1x1 | 64 | 1 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 64 | 1 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 3x3 | 256 | 2 |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 3x3 | 512 | 2 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 3x3 | 1024 | 2 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Global Average Pooling Layer | 1x1 | 1000 | 1 |
| Fully Connected Layer | 13x13x2048 | N/A | N/A |
| Fully Connected Layer | 1x4096 | N/A | N/A |
| Sortie | 13x13x45 | N/A | N/A |

Les bounding boxes sont prédites de la même façon que pour YOLO v2. La seule différence est que pour prédire la localisation d'objet à trois échelles différentes, le système extrait les caractéristiques de l'image à ces trois échelles en se basant sur un concept similaire aux feature pyramid Network. 

Pour chaque image, 3 bounding boxes sont prédites à chaque échelle. A chaque échelle, le tenseur en sortie est donc de taille 13x13x{3x(4+1+80)} car il y a 80 classes à prédire. 

**Performances du réseau** : Meilleur que YOLO précédent et SSD et ses variantes mais RetinaNet est meilleur pour la prédiction de petits objets. 

Lien vers l'article : https://pjreddie.com/media/files/papers/YOLOv3.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

https://towardsdatascience.com/yolo-v3-object-detection-

https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/

## SSD ##

**Architecture globale** : VGG-16 + 6 couches de convolution + 2 couches fully connected. 

**Architecture détaillée** : 

Lien vers le schéma de l'architecture : https://towardsdatascience.com/understanding-ssd-multibox-real-time-object-detection-in-deep-learning-495ef744fab

Lien vers le schéma de l'architecture de VGG-16 : https://medium.com/@sidereal/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5

La détection des éléments avec le réseau SSD fonctionne en deux étapes qui sont l'extraction des cartes de caractéristiques d'une image et appliquer des filtres de convolution afin de détecter l'objet. Le réseau VGG-16 au début de SSD sert à extraîre les cartes de caractéristiquers et les couches de convolution qui suivent se chargent de la détection de l'objet. Chacune des prediction contient les 4 coordonnées des bounding boxes et 21 scores pour chacune des classes sur lesquelles le réseau a été entraîné et 1 classe qui indique qu'il n'y a pas d'objet. 

Le réseau SSD n'utilise pas un réseau spécifique pour la proposition de régions. Au lieu de ça, il utilise une méthode très simple qui consiste à calculer la localisation et le score d'appartenance à chaque classe en utilisant des petits filtres de convolution à l'échelle d'une cellule sur les feature maps qui sont divisées en des grilles de tailles 8x8 pour les objets de faible résolution à des grillesd e taille 4x4 pour les objets de plus grande résolution. Par exemple, pour calculer les coordonnées de chaque bounding box et le score d'appartenance à une classe de chacune, un filtre de convolution de taille 3x3 prendra en entrée les cellules avoisinantes dans un rayon afin de former un carré englobant la cellule de taille 3x3 (cellules). Ce filtre de convolution une fois appliqué ressortira les coordonnées de chacune des bounding boxes (décalage des coordonnées de la bounding box par rapport au centre de l'image, la largeur et la hauteur) et le score d'appartenance à chacune des classes. Chaque filtre appliqué ressort donc 25 sorties. Au total, il y a 6 couches chargées de la prédiction, ce qui correspond à 8732 prédictions au total. 

**Performances du réseau** : Plus rapide et plus performant que YOLO v1. Aussi performant que les réseaux de l'état de l'art de la détection d'objet dans des images mais pas en temps réel. 

Lien article : https://arxiv.org/pdf/1512.02325.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

https://arxiv.org/pdf/1612.03144.

https://towardsdatascience.com/understanding-ssd-multibox-real-time-object-detection-in-deep-learning-

https://medium.com/@smallfishbigsea/understand-ssd-and-implement-your-own-

https://medium.com/@jonathan_hui/ssd-object-detection-single-shot-multibox-detector-for-real-time-processing-9bd8deac0e06

## DSSD ## 

Ajout d'un module de déconvolution au réseau SSD. A une meilleur mAP que SSD de 1.1%, mais la vitesse diminue de 46 fps à 12 fps.

**Architecture globale** : Residual-101 au lieu de VGG-16

Lien article : https://arxiv.org/pdf/1701.06659.pdf

Autres liens pour comprendre le fonctionnement du réseau :

https://arxiv.org/pdf/1512.03385.pdf

## FSSD ##

## ESSD ##

Lien article : https://arxiv.org/pdf/1801.05918.pdf

## RetinaNet ##

**Architecture globale** : 
- RetinaNet-50 : Backbone (FPN construit au-dessus de ResNet50 (50 couches de convolution)) suivi de deux task specific subnetworks.
- RtinaNet-101 : Backbone (FPN construit au-dessus de ResNet101 (101 couches de convolution)) suivi de deux task specific subnetworks.

**Architecture détaillée** :

Architecture de ResNet-50 : https://www.jeremyjordan.me/convnet-architectures/

La fonction de cross-entropy loss a été remplacée par une fonction de Focal loss afin d'améliorer de façon significative la précision de détection. 

*Cross-Entropy loss* : (log loss) Mesure la performance de la classification d'un model dont la sortie est une probabilité comprise entre 0 et 1. Le cross-entropy loss augmente si la probabilité prédite diverge de l'étiquette réelle. Un modèle parfait devrait avoir un log loss de 0.

*Focal loss* : Le principe est le même sauf qu'on ajoute des poids aux erreurs. Par exemple, l'écart de proba d'un objet prédit aura un poids faible tandis qu'un exemple mal classifié aura un poids fort, ce qui va donc permettre de mieux évaluer la classification d'un model. 

Les backbone network calcule une carte de convolution des caractéristiques de l'image. Le FPN permet de prédire des objets à différentes échelles. Afin de prédire des objets à différentes échelles, des anchors sont générés de la même façon que pour les RPN. A chaque position, neuf anchors sont générés, dont 3 de chaque échelle. Ces trois échelles sont 128x128, 256x256, 512x512. Les ratios des 3 anchors pour chacune des échelles sont respectivement 1:1, 1:2 et 2:1. On choisit une position à chaque stride de 16. Toutes les bounding boxes situées en dehors de l'image ne sont pas prises en compte pour la détection. A chaque niveau du FPN (P3 à P7) il y a donc 9 anchors qui sont générés. A chaque niveau, les anchors générés sont envoyés à deux subnetworks qui sont le classification subnetwork et le box regression subnetwork.

Le classification subnetwork prédit la probabilité de présence d'un objet à chacune des positions pour les 9 anchors et la proba d'appartenance de cet objet aux K classes sur lesquelles il a été entraîné. le classification subnetwork est un ensemble de couches de convolution suivi de couches fully connected afin de retourner les prédictions finales. 

Le box regression subnet lui prédit les offsets entre les ground truth boxes et les anchors prédits pour chaque anchor à chaque position. Pour chaque anchor, il y a donc 4 prédictions : décalage des coordonnées du centre, décalage de hauteur et décalage de largeur. 

**Performances du réseau** : RetinaNet est un réseau qui détecte en une étape (comme SSD et YOLO) et qui a la performances des réseaux qui détectent en deux étapes (Faster-RCNN). Plus rapide que les two-stages detectors mais moins que les autres one-stage detectors. Plus le réseau en backbone est grand plus la précision est grande mais plus le réseau est lent. 

Lien vers l'article : https://arxiv.org/pdf/1708.02002.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau : 

https://medium.com/@smallfishbigsea/notes-on-focal-loss-and-retinanet-9c614a2367c6

https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html

https://medium.com/@14prakash/the-intuition-behind-retinanet-eb636755607d

https://handong1587.github.io/deep_learning/2015/10/09/object-detection.html

https://www.analyticsvidhya.com/blog/2018/06/understanding-building-object-detection-model-python

https://medium.com/@smallfishbigsea/faster-r-cnn-explained-864d4fb7e3f8